# ⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow

**Objetivos de Aprendizaje:**
- [ ] Comprender los conceptos de DAGs (Directed Acyclic Graphs)
- [ ] Crear y configurar DAGs de Apache Airflow
- [ ] Implementar operators y tasks
- [ ] Manejar dependencias entre tareas
- [ ] Monitorear y debuggear pipelines
- [ ] Implementar buenas prácticas de orquestación

**Duración Estimada:** 120 minutos  
**Nivel de Dificultad:** Intermedio  
**Prerrequisitos:** Notebooks nivel Junior completos

---

## ⚠️ RECORDATORIO IMPORTANTE

### 🚨 NOTEBOOKS vs PRODUCCIÓN

Este curso usa notebooks para **enseñanza**, pero en tu trabajo real:

**❌ NO uses notebooks para pipelines en producción**

**✅ USA:**
- Scripts Python modulares en `src/`
- Airflow DAGs en `dags/` (archivos `.py`, NO `.ipynb`)
- Docker para containerización
- CI/CD para deployment automático
- Tests con pytest

**📖 Consulta:** `notebooks/⚠️_IMPORTANTE_LEER_PRIMERO.md` para detalles

**En este nivel aprenderás:** Cómo convertir código de notebooks a DAGs de Airflow de producción.

---

**Autor:** LuisRai (Luis J. Raigoso V.) | © 2024-2025

---

## 🎯 ¿Qué es Apache Airflow?

**Apache Airflow** es una plataforma open-source para:

✅ **Orquestar** pipelines de datos complejos  
✅ **Programar** ejecuciones automáticas  
✅ **Monitorear** el estado de los pipelines  
✅ **Manejar** dependencias entre tareas  
✅ **Reintenta r** tareas fallidas automáticamente  

### 🏗️ Conceptos Clave:

- **DAG (Directed Acyclic Graph)**: Pipeline completo definido como código
- **Operator**: Unidad de trabajo (tarea individual)
- **Task**: Instancia de un operator
- **Scheduler**: Programa la ejecución de DAGs
- **Executor**: Ejecuta las tareas
- **Web UI**: Interfaz para monitoreo

### 🌟 Ventajas de Airflow:

```
✅ Código Python nativo
✅ Pipelines como código (versionables)
✅ Rico ecosistema de operators
✅ Escalable y extensible
✅ Monitoreo visual intuitivo
✅ Manejo robusto de errores
```

## 📦 Setup y Configuración

### Instalación de Airflow

```bash
# Instalar Airflow (ejecutar en terminal)
pip install apache-airflow==2.7.0

# Inicializar base de datos
airflow db init

# Crear usuario admin
airflow users create \
    --username admin \
    --firstname Admin \
    --lastname User \
    --role Admin \
    --email admin@example.com

# Iniciar webserver
airflow webserver --port 8080

# Iniciar scheduler (en otra terminal)
airflow scheduler
```

In [ ]:
# Verificar instalación de Airflow
import sys
import subprocess

try:
    import airflow
    print(f"✅ Apache Airflow instalado")
    print(f"   Versión: {airflow.__version__}")
    print(f"   Ubicación: {airflow.__file__}")
except ImportError:
    print("❌ Apache Airflow no está instalado")
    print("\n📝 Para instalar, ejecuta en terminal:")
    print("   pip install apache-airflow==2.7.0")

# Librerías que usaremos
from datetime import datetime, timedelta
import pandas as pd
import json
import os

print("\n✅ Librerías auxiliares importadas")

### 🔧 Verificación de Entorno Airflow

**Concepto:** Airflow requiere instalación separada y configuración inicial antes de usar.

**Componentes principales:**
- **Webserver:** UI en puerto 8080 para monitoreo visual
- **Scheduler:** Daemon que programa y dispara DAGs según cron expressions
- **Metadata DB:** SQLite (dev) o PostgreSQL (prod) para estado de ejecuciones
- **Executor:** LocalExecutor, CeleryExecutor, KubernetesExecutor

**Arquitectura:**
```
DAG Files (.py) → Scheduler → Executor → Tasks → Metadata DB → Webserver UI
```

**Nota:** Este notebook usa ejemplos de código; en producción, DAGs se colocan en `~/airflow/dags/`.

## 🔨 Creando tu Primer DAG

### Estructura Básica de un DAG

In [ ]:
# Ejemplo de DAG básico (guardar como archivo .py en la carpeta dags/)
dag_basico = """
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta

# Argumentos por defecto para el DAG
default_args = {
    'owner': 'data_engineer',
    'depends_on_past': False,
    'email': ['tu_email@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2024, 1, 1),
}

# Definición del DAG
dag = DAG(
    'mi_primer_dag',
    default_args=default_args,
    description='Un DAG simple de demostración',
    schedule_interval=timedelta(days=1),
    catchup=False,
    tags=['ejemplo', 'tutorial'],
)

# Función Python simple
def imprimir_fecha(**context):
    execution_date = context['execution_date']
    print(f"Ejecutando DAG en: {execution_date}")
    return f"Completado en {execution_date}"

# Tareas del DAG
tarea_inicio = BashOperator(
    task_id='inicio',
    bash_command='echo "Iniciando pipeline..."',
    dag=dag,
)

tarea_python = PythonOperator(
    task_id='procesar_datos',
    python_callable=imprimir_fecha,
    provide_context=True,
    dag=dag,
)

tarea_fin = BashOperator(
    task_id='finalizar',
    bash_command='echo "Pipeline completado!"',
    dag=dag,
)

# Definir dependencias
tarea_inicio >> tarea_python >> tarea_fin
"""

print("📋 Ejemplo de DAG básico:")
print(dag_basico)
print("\n💾 Guarda este código en: $AIRFLOW_HOME/dags/mi_primer_dag.py")

### 📋 DAG Definition: Default Args y Configuración

**Concepto:** Un DAG es un grafo dirigido acíclico que define un pipeline completo como código Python.

**default_args:** Configuración compartida por todas las tasks del DAG:
- **owner:** Responsable del DAG (para auditoría)
- **retries:** Número de reintentos automáticos en caso de fallo
- **retry_delay:** Tiempo de espera entre reintentos
- **start_date:** Primera fecha elegible para ejecución
- **depends_on_past:** Si True, espera éxito de ejecución anterior

**Parámetros del DAG:**
- **dag_id:** Identificador único del pipeline
- **schedule_interval:** Cron expression o timedelta para frecuencia
- **catchup:** Si False, no ejecuta periodos pasados al activar DAG

**Principio:** DAGs son declarativos e idempotentes - misma ejecución produce mismo resultado.

## 🎭 Operators Principales

### 1. PythonOperator - Ejecutar Funciones Python

In [ ]:
# Ejemplo de funciones para PythonOperator
def extraer_datos(**context):
    """
    Simula extracción de datos
    """
    import random
    
    print("🔽 Extrayendo datos...")
    
    # Simular datos extraídos
    datos = [
        {'id': i, 'valor': random.randint(100, 1000), 'fecha': str(datetime.now().date())}
        for i in range(1, 11)
    ]
    
    print(f"✅ Extraídos {len(datos)} registros")
    
    # Pasar datos a la siguiente tarea usando XCom
    return datos

def transformar_datos(**context):
    """
    Transforma los datos extraídos
    """
    # Obtener datos de la tarea anterior usando XCom
    ti = context['ti']
    datos = ti.xcom_pull(task_ids='extraer')
    
    print("⚙️ Transformando datos...")
    
    # Transformación simple
    datos_transformados = [
        {
            **d,
            'valor_duplicado': d['valor'] * 2,
            'categoria': 'Alto' if d['valor'] > 500 else 'Bajo'
        }
        for d in datos
    ]
    
    print(f"✅ Transformados {len(datos_transformados)} registros")
    return datos_transformados

def cargar_datos(**context):
    """
    Carga los datos transformados
    """
    ti = context['ti']
    datos = ti.xcom_pull(task_ids='transformar')
    
    print("📤 Cargando datos...")
    
    # Guardar en archivo JSON (simulando carga a BD)
    output_path = '../datasets/processed/datos_pipeline.json'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(datos, f, indent=2)
    
    print(f"✅ Datos guardados en {output_path}")
    return output_path

# Simular ejecución
print("🧪 Simulando ejecución del pipeline:\n")

context_mock = {
    'execution_date': datetime.now(),
    'ti': type('obj', (object,), {
        'xcom_pull': lambda self, task_ids: [
            {'id': 1, 'valor': 750, 'fecha': '2024-01-01'},
            {'id': 2, 'valor': 250, 'fecha': '2024-01-01'}
        ]
    })()
}

datos_ext = extraer_datos(**context_mock)
print(f"\nDatos extraídos (muestra): {datos_ext[:2]}")

# Modificar context para transformar
context_mock['ti'].xcom_pull = lambda task_ids: datos_ext
datos_trans = transformar_datos(**context_mock)
print(f"\nDatos transformados (muestra): {datos_trans[:2]}")

# Cargar
context_mock['ti'].xcom_pull = lambda task_ids: datos_trans
ruta_salida = cargar_datos(**context_mock)
print(f"\n✅ Pipeline simulado completado")

### 🐍 PythonOperator y XCom: Compartir Datos Entre Tasks

**Concepto:** PythonOperator ejecuta funciones Python arbitrarias como tasks en el DAG.

**XCom (Cross-Communication):**
- Mecanismo para pasar datos entre tasks
- `return valor`: guarda en XCom automáticamente
- `ti.xcom_pull(task_ids='anterior')`: recupera datos de task previa
- Almacenado en metadata DB (límite ~48KB por seguridad)

**Context dict:** Inyectado automáticamente en funciones con `**context`:
- `ti` (TaskInstance): objeto para XCom y metadata
- `execution_date`: fecha lógica de ejecución
- `dag_run`: información de la ejecución del DAG
- `params`: parámetros configurables

**Buena práctica:** Para datasets grandes, usar XCom solo para paths/URLs, almacenar datos en S3/HDFS.

### 2. BashOperator - Ejecutar Comandos Shell

In [ ]:
# Ejemplos de uso de BashOperator
bash_examples = {
    'verificar_archivo': {
        'task_id': 'verificar_datos',
        'bash_command': 'test -f /path/to/data.csv && echo "Archivo existe" || echo "Archivo no encontrado"',
        'descripcion': 'Verifica que un archivo exista antes de procesarlo'
    },
    'limpiar_temp': {
        'task_id': 'limpiar_archivos',
        'bash_command': 'rm -rf /tmp/data_temp/*',
        'descripcion': 'Limpia archivos temporales'
    },
    'ejecutar_script': {
        'task_id': 'procesar_sql',
        'bash_command': 'psql -U user -d database -f /scripts/query.sql',
        'descripcion': 'Ejecuta un script SQL'
    },
    'mover_archivo': {
        'task_id': 'archivar_datos',
        'bash_command': 'mv /data/raw/{{ ds }}.csv /data/processed/',
        'descripcion': 'Mueve archivo procesado con templating'
    }
}

print("🔧 Ejemplos de BashOperator:\n")
for nombre, config in bash_examples.items():
    print(f"📌 {nombre}:")
    print(f"   Task ID: {config['task_id']}")
    print(f"   Comando: {config['bash_command']}")
    print(f"   Uso: {config['descripcion']}")
    print()

### 💻 BashOperator: Integración con Sistema Operativo

**Concepto:** Ejecuta comandos shell/bash directamente desde Airflow, útil para scripts legacy o herramientas CLI.

**Casos de uso típicos:**
- Verificar existencia de archivos antes de procesar
- Ejecutar scripts SQL con `psql` o `mysql`
- Mover/copiar archivos entre directorios
- Limpiar archivos temporales
- Ejecutar comandos de cloud CLI (aws s3 sync, gsutil)

**Jinja Templating:**
- Variables de Airflow disponibles en bash_command
- `{{ ds }}`: execution_date como YYYY-MM-DD
- `{{ dag_run.conf }}`: parámetros de ejecución manual
- `{{ var.value.my_var }}`: Airflow Variables

**Ventaja:** Reutilizar scripts bash existentes sin reescribir en Python.

## 🔗 Manejo de Dependencias

### Diferentes Formas de Definir Dependencias

In [ ]:
dependencias_ejemplos = """
# Forma 1: Usando >> (recomendado - más legible)
tarea_a >> tarea_b >> tarea_c
# Significa: A debe completarse antes de B, B antes de C

# Forma 2: Usando <<
tarea_c << tarea_b << tarea_a
# Mismo resultado que Forma 1, pero leyendo al revés

# Forma 3: set_downstream() y set_upstream()
tarea_a.set_downstream(tarea_b)
tarea_b.set_downstream(tarea_c)

# Dependencias múltiples (paralelo)
tarea_inicio >> [tarea_a, tarea_b, tarea_c] >> tarea_fin
# Significa: A, B y C se ejecutan en paralelo después de inicio

# Dependencias complejas
tarea_inicio >> tarea_a
tarea_inicio >> tarea_b
tarea_a >> tarea_c
tarea_b >> tarea_c
tarea_c >> tarea_fin
# Significa: A y B en paralelo, luego C, luego fin

# Usando listas para múltiples dependencias
[tarea_a, tarea_b] >> tarea_c >> [tarea_d, tarea_e]
"""

print("🔗 Patrones de Dependencias en Airflow:")
print(dependencias_ejemplos)

# Visualización de patrones comunes
print("\n📊 Patrones de Flujo Comunes:\n")

patrones = {
    'Lineal': """
        A → B → C → D
    """,
    'Fan-out (Paralelo)': """
             ┌→ B ┐
        A → ├→ C ├→ F
             └→ D ┘
    """,
    'Fan-in (Convergente)': """
        A ┐
        B ├→ D → E
        C ┘
    """,
    'Diamante': """
             ┌→ B ┐
        A → │     ├→ D
             └→ C ┘
    """
}

for nombre, diagrama in patrones.items():
    print(f"🔸 {nombre}:{diagrama}")

### 🔗 Dependency Management: Orquestando el Flujo

**Concepto:** Dependencias definen el orden de ejecución de tasks en el DAG.

**Operadores de dependencia:**
- `>>` (bitshift right): `A >> B` = "B depende de A"
- `<<` (bitshift left): `C << B` = "C depende de B"
- `[A, B] >> C`: C espera a que A y B completen

**Patrones de flujo:**
- **Lineal:** A → B → C (secuencial, sin paralelismo)
- **Fan-out:** A → [B, C, D] (ejecución paralela)
- **Fan-in:** [A, B, C] → D (convergencia, espera todas)
- **Diamante:** A → [B, C] → D (paralelo + convergencia)

**Trigger Rules:**
- `all_success` (default): espera que todas las upstream tasks tengan éxito
- `all_failed`: ejecuta solo si todas upstream fallan
- `one_success`: ejecuta si al menos una upstream tiene éxito
- `none_failed`: ejecuta si ninguna upstream falló (permite skipped)

**Uso:** Optimizar paralelismo maximiza throughput del pipeline.

## 🎨 DAG Completo: ETL de E-commerce

In [ ]:
# DAG completo de ejemplo para guardar en dags/
dag_ecommerce = '''
from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.bash import BashOperator
from airflow.operators.dummy import DummyOperator
from datetime import datetime, timedelta
import pandas as pd
import json
import os

default_args = {
    'owner': 'ecommerce_team',
    'depends_on_past': False,
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'etl_ecommerce_ventas',
    default_args=default_args,
    description='Pipeline ETL para procesar ventas de e-commerce',
    schedule_interval='0 2 * * *',  # Diario a las 2 AM
    start_date=datetime(2024, 1, 1),
    catchup=False,
    tags=['ecommerce', 'ventas', 'etl'],
)

# --- FUNCIONES ---

def verificar_datos_disponibles(**context):
    """Verifica si hay datos nuevos para procesar"""
    execution_date = context['execution_date']
    fecha_archivo = execution_date.strftime('%Y%m%d')
    archivo = f'/data/raw/ventas_{fecha_archivo}.csv'
    
    if os.path.exists(archivo):
        print(f"✅ Archivo encontrado: {archivo}")
        return 'extraer_ventas'
    else:
        print(f"❌ No hay datos para {fecha_archivo}")
        return 'no_data_skip'

def extraer_ventas(**context):
    """Extrae ventas desde archivos CSV"""
    execution_date = context['execution_date']
    fecha_archivo = execution_date.strftime('%Y%m%d')
    
    print(f"🔽 Extrayendo ventas del {fecha_archivo}")
    
    # Simular lectura
    df = pd.DataFrame({
        'venta_id': range(1, 101),
        'producto': ['Laptop', 'Mouse', 'Teclado'] * 33 + ['Monitor'],
        'cantidad': [1, 2, 1] * 33 + [1],
        'precio': [1000, 25, 75] * 33 + [300],
        'fecha': execution_date
    })
    
    print(f"✅ Extraídas {len(df)} ventas")
    return df.to_json()

def transformar_ventas(**context):
    """Transforma y enriquece datos de ventas"""
    ti = context['ti']
    ventas_json = ti.xcom_pull(task_ids='extraer_ventas')
    df = pd.read_json(ventas_json)
    
    print("⚙️ Transformando datos de ventas")
    
    # Calcular total
    df['total'] = df['cantidad'] * df['precio']
    
    # Categorizar por monto
    df['categoria_venta'] = df['total'].apply(
        lambda x: 'Premium' if x >= 500 else 'Standard'
    )
    
    # Limpiar duplicados
    df = df.drop_duplicates(subset=['venta_id'])
    
    print(f"✅ Transformados {len(df)} registros")
    return df.to_json()

def calcular_metricas(**context):
    """Calcula métricas agregadas"""
    ti = context['ti']
    ventas_json = ti.xcom_pull(task_ids='transformar_ventas')
    df = pd.read_json(ventas_json)
    
    metricas = {
        'total_ventas': len(df),
        'ingreso_total': df['total'].sum(),
        'ticket_promedio': df['total'].mean(),
        'producto_mas_vendido': df['producto'].mode()[0],
        'fecha_proceso': context['execution_date'].isoformat()
    }
    
    print(f"📊 Métricas calculadas: {metricas}")
    return metricas

def cargar_warehouse(**context):
    """Carga datos en el data warehouse"""
    ti = context['ti']
    ventas_json = ti.xcom_pull(task_ids='transformar_ventas')
    
    print("📤 Cargando datos al warehouse...")
    # Aquí iría la conexión real a BD
    # connection.execute("INSERT INTO ventas ...")
    
    print("✅ Datos cargados exitosamente")

def enviar_notificacion(**context):
    """Envía notificación de finalización"""
    ti = context['ti']
    metricas = ti.xcom_pull(task_ids='calcular_metricas')
    
    mensaje = f"""
    Pipeline ETL Completado
    -----------------------
    Fecha: {metricas['fecha_proceso']}
    Total ventas: {metricas['total_ventas']}
    Ingreso total: ${metricas['ingreso_total']:,.2f}
    Ticket promedio: ${metricas['ticket_promedio']:.2f}
    """
    
    print(f"📧 Enviando notificación:\n{mensaje}")

# --- TAREAS ---

inicio = DummyOperator(
    task_id='inicio',
    dag=dag,
)

verificar = BranchPythonOperator(
    task_id='verificar_datos',
    python_callable=verificar_datos_disponibles,
    dag=dag,
)

no_data = DummyOperator(
    task_id='no_data_skip',
    dag=dag,
)

extraer = PythonOperator(
    task_id='extraer_ventas',
    python_callable=extraer_ventas,
    dag=dag,
)

transformar = PythonOperator(
    task_id='transformar_ventas',
    python_callable=transformar_ventas,
    dag=dag,
)

metricas = PythonOperator(
    task_id='calcular_metricas',
    python_callable=calcular_metricas,
    dag=dag,
)

cargar = PythonOperator(
    task_id='cargar_warehouse',
    python_callable=cargar_warehouse,
    dag=dag,
)

notificar = PythonOperator(
    task_id='enviar_notificacion',
    python_callable=enviar_notificacion,
    dag=dag,
)

fin = DummyOperator(
    task_id='fin',
    trigger_rule='none_failed_min_one_success',
    dag=dag,
)

# --- FLUJO ---
inicio >> verificar >> [extraer, no_data]
extraer >> transformar >> [metricas, cargar]
[metricas, cargar] >> notificar >> fin
no_data >> fin
'''

print("📋 DAG Completo de E-commerce:")
print("\n💾 Guarda este código en: $AIRFLOW_HOME/dags/etl_ecommerce_ventas.py")
print("\n🎯 Características del DAG:")
print("   • Branching (decisiones condicionales)")
print("   • Procesamiento paralelo")
print("   • XCom para pasar datos entre tareas")
print("   • Manejo de escenarios sin datos")
print("   • Notificaciones automáticas")
print("   • Trigger rules personalizados")

### 🏗️ DAG Completo: Patrón ETL Productivo

**Concepto:** DAG que implementa pipeline ETL completo con branching, paralelismo y notificaciones.

**Componentes avanzados:**
- **DummyOperator:** Tareas placeholder para organización lógica
- **BranchPythonOperator:** Ejecución condicional basada en lógica
- **Trigger rules:** Control fino de cuándo ejecutar tasks downstream

**Flujo del DAG:**
1. **Inicio** → Verificar datos disponibles
2. **Branch:** Si hay datos → Extraer | Si no → Skip
3. **Paralelo:** Transformar + Calcular métricas
4. **Convergencia:** Cargar al warehouse
5. **Notificación:** Enviar resumen de ejecución

**Patrón Branch:**
```python
def verificar(**context):
    if condicion:
        return 'tarea_si'
    return 'tarea_no'
```

**Producción:** Este patrón es base para pipelines enterprise con alerting y observabilidad.

## 🎓 Resumen y Mejores Prácticas

### ✅ Mejores Prácticas:

1. **Idempotencia**: Las tareas deben producir el mismo resultado si se ejecutan múltiples veces
2. **Atomicidad**: Cada tarea debe ser una unidad indivisible de trabajo
3. **No estado compartido**: Usar XCom para comunicación entre tareas
4. **Logging apropiado**: Registrar información útil para debugging
5. **Manejo de errores**: Implementar retries y alertas
6. **Documentación**: Docstrings claros y tags descriptivos
7. **Testing**: Probar DAGs antes de producción

### 🔜 Próximos Temas:

- Sensors y event-driven workflows
- TaskGroups para organización
- Dynamic DAGs
- Integración con Cloud (AWS, GCP, Azure)
- Monitoreo y alertas avanzadas

---

**¡Has dominado los fundamentos de Apache Airflow!** 🚀

---

## 🧭 Navegación

**← Anterior:** [← README del Curso](../../README.md)

**Siguiente →:** [Streaming con Apache Kafka: Fundamentos →](02_streaming_kafka.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb) ← 🔵 Estás aquí
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb)
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb)
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb)
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
